In [ ]:
pip install -q llama-index llama-index-retrievers-bedrock boto3 botocore

In [ ]:
from os import environ

environ["AWS_DEFAULT_REGION"] = ""
environ["AWS_ACCESS_KEY_ID"] = ""
environ["AWS_SECRET_ACCESS_KEY"] = ""
environ["AWS_SESSION_TOKEN"] = ""
aws_key = ""

In [2]:
questions = [
    {
        "question": "Which expert has the 'User Acceptance Testing' skill?",
        "expected_answer": "Lee Smith (1043668)",
    },
    {
        "question": "What languages Lisa Mills speaks?",
        "expected_answer": "German, English (1045114)",
    },
    {
        "question": "Who wrote 'HEMA Reimbursement Strategy &  Preparation'?",
        "expected_answer": "Amber Logan(172949)",
    },
    {
        "question": "Who has the 'Breast Cancer' skill?",
        "expected_answer": "Anthony Odonnell(21850), Courtney Spears(125846), Michelle Marshall(127234), Jason Mendoza(163541), Allison Robles(164561), Antonio Smith(169629), Tiffany Smith(171759)",
    },
    {
        "question": "Who is located in New Zeland?",
        "expected_answer": "Kelly Velasquez(168994), Lee Smith(1043668), Peter Nelson(1083261)",
    },
]

In [ ]:
from llama_index.retrievers.bedrock import AmazonKnowledgeBasesRetriever
from langchain_openai import ChatOpenAI

KB_ID = "HVW4QBSXJN"
retrieved_results = []

retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=KB_ID,
    retrieval_config={
        "vectorSearchConfiguration": {
            "numberOfResults": 4,
        }
    },
)

llm = ChatOpenAI(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    base_url="https://us.aigw.galileo.roche.com/v1",
    api_key=aws_key,
    temperature=0.0,
)

for question in questions:
    result = retriever.retrieve(question["question"])
    llm_response = llm.invoke(f"""Answer the question based only on the context provided.

    Context: {result[0].get_content()}

    Question: {question["question"]}""")

    object = {
        "question": question["question"],
        "expected_answer": question["expected_answer"],
        "result": result[0].get_content(),
        "llm_response": llm_response.content,
    }
    retrieved_results.append(object)


retrieved_results